# Motif Enrichment
- The motif hits matrix save the max loglikelyhood score of a motif in that DMR
- Before run the motif enrichment, I filter the scores for each motif using cutoff defined by
    - `np.quantile(motif_score[motif_score > 0]) * mask_quantile_to99`
- This filter will remove hits with low score, which is likely to be false positive
- Its likely boosting the significance of real enrichment

In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
from scipy.sparse import csr_matrix
from statsmodels.stats.multitest import multipletests
import pathlib

In [2]:
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = 'IT-L6'

motif_hits_path = f'/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/DMRAnalysis/AggLayerDMR/{group}/MotifScan.h5ad'
output_dir = f'/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/DMRAnalysis/AggLayerDMR/{group}/MotifEnrichment'
dmr_hits_path = f'/home/hanliu/project/mouse_rostral_brain/study/ITSpatial/DMRAnalysis/AggLayerDMR/{group}/{group}.total.bed'


In [3]:
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## DMR hits

In [4]:
dmr_bed_df = pd.read_csv(dmr_hits_path, sep='\t', header=None, index_col=3)
dmr_bed_df.head()

,0,1,2
3,,,
IT-L6_0,chr1,3067000,3067002
IT-L6_1,chr1,3094121,3094121
IT-L6_2,chr1,3113766,3113890
IT-L6_3,chr1,3121819,3121821
IT-L6_4,chr1,3144003,3144003


In [5]:
motif_hits = anndata.read_h5ad(motif_hits_path)
motif_hits = motif_hits[dmr_bed_df.index, :].copy()
motif_ids = motif_hits.var_names
motif_hits.X = motif_hits.X.todense()
motif_hits

AnnData object with n_obs × n_vars = 219056 × 719 
    obs: 'chrom', 'start', 'end'

In [6]:
# mask small motif scores
motif_cutoff = pd.Series(np.apply_along_axis(lambda i: np.quantile(i[i>0], 0.99) * mask_quantile_to99, 
                                             0, motif_hits.X),
                         index=motif_hits.var_names)
# only keep value larger than the cutoff for each motif
motif_hits.X = np.multiply(motif_hits.X, (motif_hits.X > motif_cutoff.values[None, :]))

## Background Hits

In [7]:
background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/BackgroundDMR/MotifScan.h5ad'
)

In [8]:
dmr_bed = pybedtools.BedTool().from_dataframe(dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

In [9]:
# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, motif_ids].copy()

In [10]:
judge_matrix = csr_matrix(background_motif_hits.X > motif_cutoff.values[None, :])
background_motif_hits.X = background_motif_hits.X.multiply(judge_matrix)

In [11]:
background_motif_hits

AnnData object with n_obs × n_vars = 298533 × 719 
    obs: 'chrom', 'start', 'end'

## Motif hits contingency table

In [12]:
# calculate motif occurence, not considering hits here
pos = (motif_hits.X > 0).sum(axis=0)
pos_total = motif_hits.shape[0]

neg = (background_motif_hits.X > 0).sum(axis=0)
neg_total = background_motif_hits.shape[0]

In [13]:
tables = {}
for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
    table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
    tables[motif] = table

In [14]:
results = {}
with ProcessPoolExecutor(20) as executor:
    fs = {}
    for motif, t in tables.items():
        f = executor.submit(stats.fisher_exact, t, alternative='greater')
        fs[f] = motif

    for f in as_completed(fs):
        motif = fs[f]
        odds, p = f.result()
        results[motif] = {'oddsratio': odds, 'p_value': p}
motif_enrich_df = pd.DataFrame(results).T

_, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
motif_enrich_df['adj_p'] = p

motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
    -np.inf, -300)

records = {}
for motif, t in tables.items():
    tp, tn = t[0]
    fp, fn = t[1]
    tp_rate = tp / pos_total
    fp_rate = fp / neg_total
    records[motif] = dict(tp=tp,
                          tn=tn,
                          fp=fp,
                          fn=fn,
                          tp_rate=tp_rate,
                          fp_rate=fp_rate)
counts = pd.DataFrame(records).T
motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
motif_enrich_df['Group'] = group

In [16]:
# final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

50

In [17]:
motif_enrich_df.to_msgpack(output_dir / f'{group}.motif_enrichment.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [18]:
pybedtools.cleanup()

In [19]:
filtered_motif_df

,oddsratio,p_value,adj_p,-lgp,tp,tn,fp,fn,tp_rate,fp_rate,Group
MA0019.1,2.690450,7.848682e-171,1.710061e-169,168.766988,2203.0,216853.0,1123.0,297410.0,0.010057,0.003762,IT-L6
MA0089.2,1.791183,1.050583e-35,1.425225e-34,33.846117,1044.0,218012.0,796.0,297737.0,0.004766,0.002666,IT-L6
MA0099.3,2.087274,2.050461e-274,7.759376e-273,272.110173,5754.0,213302.0,3809.0,294724.0,0.026267,0.012759,IT-L6
MA0150.2,1.625747,6.091468e-35,8.110677e-34,33.090943,1403.0,217653.0,1179.0,297354.0,0.006405,0.003949,IT-L6
MA0461.2,2.661927,0.000000e+00,0.000000e+00,300.000000,4875.0,214181.0,2531.0,296002.0,0.022255,0.008478,IT-L6
MA0462.2,2.403852,0.000000e+00,0.000000e+00,300.000000,11087.0,207969.0,6477.0,292056.0,0.050613,0.021696,IT-L6
MA0476.1,2.316070,0.000000e+00,0.000000e+00,300.000000,13421.0,205635.0,8182.0,290351.0,0.061267,0.027407,IT-L6
MA0477.2,2.191856,3.087025e-289,1.387232e-287,286.857851,5453.0,213603.0,3437.0,295096.0,0.024893,0.011513,IT-L6
MA0478.1,1.989989,8.013011e-316,4.115254e-314,313.385603,7446.0,211610.0,5187.0,293346.0,0.033991,0.017375,IT-L6
MA0489.1,2.043048,5.260595e-237,1.454757e-235,234.837210,5207.0,213849.0,3516.0,295017.0,0.023770,0.011778,IT-L6
